<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/tcc_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-pinecone pinecone-notebooks requests langchain-groq langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 628.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58

## Configuração de Variaveis

In [2]:
from google.colab import userdata

class Settings:
  HF_TOKEN          = userdata.get('HF_TOKEN')
  PINECONE_API_KEY  = userdata.get('PINECONE_API_KEY')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')


settings = Settings()

### Geração de embeddings e indíces do banco vetorial

In [3]:
class EmbeddingModelSpecs:
  def __init__(self):
    self.name      = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
    self.dimension = 512

embeddings_model = EmbeddingModelSpecs()


In [4]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=settings.PINECONE_API_KEY)

In [5]:
import time

index_name = "index-ppc-markdown-ids-metadata-in-content"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

## Vector store por Inference API do Hugging Face (lento)

In [ ]:
# import requests
# from langchain_pinecone import PineconeVectorStore

# class CustomEmbedding:
#     def __init__(self):
#         self.hf_token = settings.HF_TOKEN
#         self.model_id = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
#         self.api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{self.model_id}"
#         self.headers = {"Authorization": f"Bearer {self.hf_token}"}

#     def embed_query(self, query):
#         return self.get_embeddings([query])[0]

#     def embed_documents(self, documents):
#         return self.get_embeddings(documents)

#     def get_embeddings(self, texts):
#         response = requests.post(self.api_url, headers=self.headers, json={"inputs": texts, "options": {"wait_for_model": True}})
#         response.raise_for_status()  # Raise an error for bad responses
#         return response.json()

# def initialize_vector_store(index):
#     # Initialize the custom embedding class
#     custom_embedding = CustomEmbedding()

#     # Initialize PineconeVectorStore with the custom embedding class
#     vector_store = PineconeVectorStore(index=index, embedding=custom_embedding)

#     return vector_store


In [ ]:
# vector_store = initialize_vector_store(index)
# results = vector_store.similarity_search("Limite de Atividades de ensino", k=2)

## Vector store local

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model.name)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [7]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

## Geração da resposta

In [39]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq



llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    groq_api_key=settings.GROQ_API_KEY,
    max_retries=4
)



def query_transformation(query):

    query_transformation_llm = ChatGroq(
        model="gemma2-9b-it",
        temperature=0,
        groq_api_key=settings.GROQ_API_KEY,
        max_retries=2
    )

    prompt = f"""
    Esse é minha pergunta: {query}

    Analise a pergunta e siga as seguintes regras:

    1. Se a pergunta for sobre dados que parecem ser tabulares, como ementas, limite de horas, fichas de avaliação, atividades ou componentes curriculares, transforme a pergunta em uma string concisa usando a estrutura de tabela markdown. Exemplo: "Qual a ementa de Teoria da Computação?" deve ser transformada para "TEORIA DA COMPUTAÇÃO | EMENTA".

    2. Se a pergunta for mais direta e não se enquadrar na categoria acima, apenas adapte a pergunta para uma versão mais concisa, sem transformá-la em uma estrutura de tabela markdown.

    3. Desfaça siglas antes de retornar a string, da seguinte maneira:
       - PPC = Plano Pedagógico do Curso
       - ACC = Atividades Curriculares Complementares
       - ORG = Organização de Computadores
       - BD = Banco de Dados
       - ENG = Engenharia
       - PGP = Planejamento e Gestão de Projetos
       - TCC = Trabalho de Conclusão de Curso

    4. Não coloque o termo em siglas, caso venha completo.

    Retorne apenas a string sem nenhuma frase introdutória.
    """

    prompt_template = ChatPromptTemplate.from_messages([("human", prompt)])

    chain = prompt_template | query_transformation_llm
    response = chain.invoke({})
    return response.content


def format_chunk(chunk):
  return chunk.split('</metadata>')[1]



def get_context(query, source_name = 'ppc2024', ids = []):

    results = vector_store.search(
        query,
        "similarity",
        k=4,
        filter={"source": source_name},
    )


    page_content = '-' * 20 + '\n\n'.join([format_chunk(result.page_content) for result in results if result.metadata['id'] not in ids])
    ids = [result.metadata['id'] for result in results if result.metadata['id'] not in ids]
    return page_content, ids



def build_prompt(contexto, pergunta):
    # Cria o prompt para o contexto do PPC
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Você é um assistente especializado no Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS. "
                "Sua função é fornecer respostas precisas e confiáveis sobre qualquer questão relacionada ao PPC, utilizando apenas as informações contidas no documento como referência. "
                "Se a resposta para pergunta não estiver nos trechos fornecidos, simplesmente informe que a resposta não foi encontrada."
                "\n\n"
                "Responda exclusivamente em português brasileiro e formate a resposta em markdown simples."
            ),
            (
                "human",
                f"Dado os seguintes trechos do PPC: {contexto}, responda a pergunta: {pergunta}.\n\n.Lembre-se de utilizar apenas o PPC como fonte para responder. Respire fundo e valide a resposta. "
            ),
        ]
    )
    return prompt



def get_completion(prompt):
    chain = prompt | llm

    response = chain.invoke({})
    print(response)
    return response


def get_answer(query, source_name):
    expanded_query = query_transformation(query)
    print('Query expandida: ', expanded_query)

    query_context, ids = get_context(query, source_name)
    print(len(ids))
    expanded_query_context, new_ids = get_context(expanded_query, source_name, ids)
    print(len(new_ids))
    context = query_context + '\n\n' + expanded_query_context

    prompt = build_prompt(context, query)
    response = get_completion(prompt)


    return context, response.content, expanded_query


In [38]:
context, answer, _ = get_answer('Qual é a ementa de Teoria da computação?', 'ppc2024')
answer

Query expandida:  TEORIA DA COMPUTAÇÃO | EMENTA 

4
4
content='A ementa da disciplina de Teoria da Computação, conforme o Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS, é a seguinte:\n\n- Máquinas de Turing e variantes.\n- A tese de Church-Turing.\n- Computabilidade.\n- Decidibilidade.\n- Redutibilidade.\n- Teoria da Complexidade.\n- Complexidade de Tempo.\n- As classes P, NP e NP-Completude.\n- Complexidade de Espaço.\n- Tópicos avançados em Teoria da Computação.\n\nFonte: PPC de Ciência da Computação da UFFS.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 2227, 'total_tokens': 2394, 'completion_time': 0.269998236, 'prompt_time': 0.111005518, 'queue_time': 0.0010307640000000035, 'total_time': 0.381003754}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None} id='run-d9d86360-235e-4729-941f-8b8cbccdc07d-0' usage_metadata={'input_tokens': 2227, 'output

'A ementa da disciplina de Teoria da Computação, conforme o Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS, é a seguinte:\n\n- Máquinas de Turing e variantes.\n- A tese de Church-Turing.\n- Computabilidade.\n- Decidibilidade.\n- Redutibilidade.\n- Teoria da Complexidade.\n- Complexidade de Tempo.\n- As classes P, NP e NP-Completude.\n- Complexidade de Espaço.\n- Tópicos avançados em Teoria da Computação.\n\nFonte: PPC de Ciência da Computação da UFFS.'

## Perguntas

In [10]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
!mkdir -p data
!cp /content/drive/My\ Drive/tcc/validacao/GroundTruthPPC2018_Diretrizes_Refinadas.csv data/
!cp /content/drive/My\ Drive/tcc/validacao/GroundTruthPPC2024_Diretrizes_Refinadas.csv data/


In [14]:
import pandas as pd
import csv
import numpy as np
from time import sleep

def process_csv_and_generate_answers(input_csv_path, output_csv_path, inference_config, source_name = 'ppc2024'):


    df = pd.read_csv(input_csv_path)

    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = ["Pergunta", "RespostaIdeal", "Diretriz_de_Avaliacao", "RespostaGerada", "Contexto", "ConsultaExpandida", "ConfiguracaoDeInferencia"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()

        for index, row in df.iterrows():
            question = row['Pergunta']

            goal_response = row['Resposta'] if not pd.isna(row['Resposta']) else ''
            evaluation_guideline = row['Diretriz_de_Avaliacao'] if not pd.isna(row['Diretriz_de_Avaliacao']) else ''

            context, response, expanded_query = get_answer(question, source_name)

            writer.writerow({
                "Pergunta": question,
                "RespostaIdeal": goal_response,
                "Diretriz_de_Avaliacao": evaluation_guideline,
                "RespostaGerada": response,
                "ConsultaExpandida": expanded_query,
                "Contexto": context,
                "ConfiguracaoDeInferencia": inference_config
            })

            print(f"Pergunta: {question}")
            print(f"Resposta: {response}")
            sleep(30)

    print(f"Novo CSV salvo com sucesso em {output_csv_path}.")


In [40]:
# Exemplo de uso
ppc_2018_input_csv_path = 'data/GroundTruthPPC2018_Diretrizes_Refinadas.csv'
ppc_2018_output_csv_path = 'QA_ppc2018.csv'
inference_config = "Mixtral e Gemma 9b(Transformação). Contexto: 4, 4 --> Original e expandida. PINECONE COM METADATA"

process_csv_and_generate_answers(ppc_2018_input_csv_path, ppc_2018_output_csv_path, inference_config, 'ppc2018')

Query expandida:  INFORMATICA BASICA | EMENTA 

4
2
content='A ementa do componente curricular Informática Básica (GEX208) do Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS é: "Fundamentos de informática. Conhecimentos de sistemas operacionais. Utilização da rede mundial de computadores. Ambientes virtuais de aprendizagem. Conhecimentos de softwares de produtividade para criação de projetos educacionais e/ou técnicos e/ou multimediáticos."' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 4951, 'total_tokens': 5085, 'completion_time': 0.216278447, 'prompt_time': 0.244908471, 'queue_time': 0.0018720870000000223, 'total_time': 0.461186918}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None} id='run-1a459128-e9ff-431c-94df-3f487754c7e4-0' usage_metadata={'input_tokens': 4951, 'output_tokens': 134, 'total_tokens': 5085}
Pergunta: Qual é a ementa de Informát

In [41]:
# Exemplo de uso
ppc_2024_input_csv_path = 'data/GroundTruthPPC2024_Diretrizes_Refinadas.csv'
ppc_2024_output_csv_path = 'QA_ppc2024.csv'
inference_config = "Mixtral e Gemma 9b(Transformação). Contexto: 4, 4 --> Original e expandida. PINECONE COM METADATA"

process_csv_and_generate_answers(ppc_2024_input_csv_path, ppc_2024_output_csv_path, inference_config, 'ppc2024')

Query expandida:  INTRODUÇÃO À COMPUTAÇÃO | EMENTA 

4
4
content='A ementa da disciplina Introdução à Computação no Curso de Ciência da Computação da UFFS é:\n\n"Introdução ao Curso de Ciência da Computação e à Universidade Federal da Fronteira Sul: principais estruturas e regulamentos. Introdução à área da Computação: inserção e desenvolvimento profissional, carreira acadêmica e técnica. Conhecimentos e comandos básicos em sistemas operacionais abertos. Fundamentos básicos da Computação: história, principais componentes de hardware e software, sistemas de numeração, aritmética binária e suas operações. Principais ferramentas e tecnologias para o desenvolvimento e versionamento de código."\n\nEssa informação pode ser encontrada no primeiro trecho do Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS fornecido.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 235, 'prompt_tokens': 3343, 'total_tokens': 3578, 'completion_time': 0.382076173, 'pro

In [42]:
import shutil
from datetime import datetime
import pytz

def save_csv_to_drive(csv_file_path, drive_folder_path):
    # Define the timezone for BRT
    brt_timezone = pytz.timezone('America/Sao_Paulo')

    file_name = csv_file_path.split('.csv')[0]

    # Generate a timestamp in BRT timezone
    timestamp = datetime.now(brt_timezone).strftime('%d_%m_%Y_%H-%M-%S')

    # Define the path in Google Drive with timestamp
    drive_path = f'{drive_folder_path}/{file_name}_{timestamp}.csv'

    # Copy the CSV file to Google Drive
    shutil.copy(csv_file_path, drive_path)
    print(f'File saved to: {drive_path}')



In [43]:
csv_file_path = 'QA_ppc2018.csv'
drive_folder_path = '/content/drive/MyDrive/tcc/resultados'
save_csv_to_drive(csv_file_path, drive_folder_path)

File saved to: /content/drive/MyDrive/tcc/resultados/QA_ppc2018_06_10_2024_14-58-21.csv


In [44]:
save_csv_to_drive('QA_ppc2024.csv', drive_folder_path)

File saved to: /content/drive/MyDrive/tcc/resultados/QA_ppc2024_06_10_2024_14-58-28.csv
